# Runge-Kutta integration for multiple coupled variables

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define coupled derivatives to integrate

In [ ]:
def dfdx(x, f, nv):
    
    #set the derivatives
    
    #equation is d^2y/dx^2 = -y
    
    #so dydx = z and dzdx = -y
    
    y = f[0]
    z = f[1]
    
    #declare an array
    y_derivs = np.zeros(nv)
    
    #set dydx = z
    y_derivs[0] = z
    
    #set dzdx = -y
    y_derivs[1] = -1 * y
    
    #here we have to return an array
    return y_derivs

## Define the 4th order RK method for multiple variables

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h, f):
    
    #dydx is the function of derivatives
    #xi is the value of x at step i
    #yi is the array of variables at step i
    #nv is the number of variables
    #h is the step size
    
    #adaptive step-size control constant arrays
    #an array for ci
    ci = np.array([0., 0., (1./5.), (3./10.), (3./5.), (1.), (7./8.)])

    #an array for aij
    aij = np.zeros(42).reshape(7, 6)
    aij[2][1] = 1./5.
    aij[3][1] = 3./40.
    aij[4][1] = 3./10.
    aij[5][1] = -11./54.
    aij[6][1] = 1631./55296.
    aij[3][2] = 9./40.
    aij[4][2] = -9./10.
    aij[5][2] = 5./2.
    aij[6][2] = 175./512.
    aij[4][3] = 6./5.
    aij[5][3] = -70./27.
    aij[6][3] = 575./13824.
    aij[5][4] = 35./27.
    aij[6][4] = 44275./110592.
    aij[6][5] = 253./4096.

    #an array for bi
    bi = np.array([0., (37./378.), 0., (250./621.), (125./594.), 0., 512./1771.])

    #an array for bis
    bis = np.array([0., 2825./27648., 0., 18575./48384., 13525./55296., 277./14336., 1./4.])

    #define k array
    ki = np.zeros( (7, nv) )
    
    #compute ki
    for i in range(1, 7):
        
        #xn + 1 for i
        xn = xi + ci[i]*h
        
        #temp y
        yn = yi.copy()
        for j in range(1, i):
            yn += aij[i, j] * ki[j, :]
            
        #k
        ki[i, :] = h * f(xn, yn, nv)
        
    #get yipo, yipo*
    yipo = yi.copy()
    yipos = yi.copy()
    
    for i in range(1, 7):
        
        yipo += bi[i] * ki[i, :]
        yipos += bis[i] * ki[i, :]

    Delta = np.fabs(yipo - yipos)
    
    return yipo, Delta

In [ ]:
'''def rk4_mv_core_s(dydx, xi, yi, nv, h):
    
    #dydx is the function of derivatives
    #xi is the value of x at step i
    #yi is the array of variables at step i
    #nv is the number of variables
    #h is the step size
    
    #adaptive step-size control constant arrays
    #an array for ci
    ci = np.array([0, 0, (1/5), (3/10), (3/5), (1), (7/8)])

    #an array for aij
    aij = np.zeros(42).reshape(7, 6)
    aij[2][1] = 1/5
    aij[3][1] = 3/40
    aij[4][1] = 3/10
    aij[5][1] = -11/54
    aij[6][1] = 1631/55296
    aij[3][2] = 9/40
    aij[4][2] = -9/10
    aij[5][2] = 5/2
    aij[6][2] = 175/512
    aij[4][3] = 6/5
    aij[5][3] = -70/27
    aij[6][3] = 575/13824
    aij[5][4] = 35/27
    aij[6][4] = 44275/110592
    aij[6][5] = 253/4096

    #an array for bi
    bi = np.array([0, (37/378), 0, (250/621), (125/594), 0, 512/1771])

    #an array for bis
    bis = np.array([0, 2825/27648, 0, 18575/48384, 13525/55296, 277/14336, 1/4])
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)    
    k6 = np.zeros(nv)
    
    #define x at 1/2 step (x_ipof - i plus one half)
#    x_ipoh = xi + 0.5*h
    
    #define x at 1 setp (x_ipo - i plus one)
#    x_ipo = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi, yi, nv)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + aij[2][1]*k1[:]
    x_temp = xi + ci[2]*h
    y_derivs = dydx(x_temp, y_temp, nv)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + aij[3][1]*k1[:] + aij[3][2]*k2[:]
    x_temp = xi + ci[3]*h
    y_derivs = dydx(x_temp, y_temp, nv)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + aij[4][1]*k1[:] + aij[4][2]*k2[:] + aij[4][3]*k3[:]
    x_temp = xi + ci[4]*h
    y_derivs = dydx(x_temp, y_temp, nv)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi[:] + aij[5][1]*k1[:] + aij[5][2]*k2[:] + aij[5][3]*k3[:] + aij[5][4]*k4[:]
    x_temp = xi + ci[5]*h
    y_derivs = dydx(x_temp, y_temp, nv)
    k4[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] + aij[6][1]*k1[:] + aij[6][2]*k2[:] + aij[6][3]*k3[:] + aij[6][4]*k4[:] + aij[6][5]*k5[:]
    x_temp = xi + ci[6]*h
    y_derivs = dydx(x_temp, y_temp, nv)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipos = yi + bis[1]*k1 + bis[2]*k2 + bis[3]*k3 + bis[4]*k4 + bis[5]*k5 + bis[6]*k6
    
    return yipos
'''

## Define an adaptive step size driver for RK4

In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 1000
    
    #set an iteration variable
    i = 0
    
    #remember the step 
    h_step = h
    
    #create an error
    Delta = np.full(nv, 2*tol)
    
    #adjust step
    while(Delta.max()/tol > 1.0):

        #yipo and error estimate
        yipo, Delta = rk4_mv_core(dydx, x_i, y_i, nv, h_step, dfdx)
        
        #if the error is too large, take a smaller step
        if((Delta.max()/tol) > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i >= imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        #iterate
        i += 1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return yipo, h_new, h_step

## Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx, a, b, y_a, tol):
    
    #dfdx - derivative wrt x
    #a - lower bound
    #b - upper bound
    #y_a - the boundary conditions
    #tol - the tolerance for integrating y
    
    #define starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an interation variable
    i = 0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1, a)
    y = np.full((1, nv), y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i + 1
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h ,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi + h_step > b):
            
            #take a smaller step
            h = b - xi
            
            #recalculate y_i + 1
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi = yi_new.copy()
        
        #add the step to the arrays
        x = np.append(x, xi)
        yi_new = np.zeros((len(x), nv))
        yi_new[0: len(x)-1, :] = y[:]
        yi_new[-1, :] = yi[:]
        del y
        y = yi_new
        
        #prevent too many iterations
        if(i >= imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        #stops loop if new xi is == b
        if(xi == b):
            flag = 0
    
    #return the answer
    return x, y

## Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

nv = 2
y_0 = np.zeros(nv)
y_0[0] = 0.0
y_0[1] = 1.0 

tolerance = 1.0e-6

#perform the integration
x, y = rk4_mv(dydx, a, b, y_0, tolerance)

## Plot the Result

In [ ]:
plt.plot(x, y[:, 0], 'o', label = 'y(x)')
plt.plot(x, y[:, 1], 'o', label = 'dydx(x)')
xx = np.linspace(0, 2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label = 'sinx')
plt.plot(xx, np.cos(xx), label = 'cosx')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend()
plt.show()

## Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0] - sine)
dydx_error = (y[:,1] - cosine)

plt.plot(x, y_error, label = "y(x) Error")
plt.plot(x, dydx_error, label = "dydx(x) Error")
plt.legend()